# Experiment MNIST - TSNE

This experiment tries to replicate the reult obtained by the paper https://arxiv.org/pdf/1906.00722.pdf where a process of dimensionality reduction was applied on the mnist dataset, and values of 0.946 for Trustworthiness and 0.938 for continuity were obtained. 

## Basic imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pandas as pd
from pathlib import Path
print(sys.path)
sys.path.append("../../")

['/home/hubert/librep-hiaac/experiments/Topological_ae', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages']


## Loading the dataset

In [3]:
import tensorflow as tf
from librep.transforms import TSNE
from librep.transforms import UMAP
from librep.datasets.multimodal import TransformMultiModalDataset, ArrayMultiModalDataset, WindowedTransform
from librep.metrics.dimred_evaluator import DimensionalityReductionQualityReport, MultiDimensionalityReductionQualityReport
from librep.datasets.har.loaders import MNISTView

2022-10-06 01:11:36.982952: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-06 01:11:36.983031: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
loader = MNISTView("../../data/old-views/MNIST/default/", download=False)
train_val_mnist, test_mnist = loader.load(concat_train_validation=True)

In [5]:
train_val_mnist, test_mnist

(PandasMultiModalDataset: samples=60000, features=785, no. window=1,
 PandasMultiModalDataset: samples=10000, features=785, no. window=1)

In [6]:
train_val_mnist.data

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train_val_mnist.data.iloc[:,1:]

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
transform_umap = UMAP()
umap_transform = WindowedTransform(
   transform=UMAP(), fit_on="all", transform_on="all")

transformer = TransformMultiModalDataset(transforms=[umap_transform])
train_applied_umap = transformer(train_val_mnist)
test_applied_umap = transformer(test_mnist)
assert 1==0

In [ ]:
transform_tsne = TSNE()
train_tsne = train_val_mnist.data.iloc[:,1:]
print(train_val_mnist.shape)
assert 1==0
# transformer = TransformMultiModalDataset(transforms=[transform_tsne])
# train_val_mnist_tsne = transformer(train_val_mnist)
# test_mnist_tsne = transformer(test_mnist)

In [ ]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=500)
metrics_train_applied_tsne = metrics_reporter.evaluate([mnist_dataset_train, train_applied_tsne])

In [ ]:
list(range(10))

In [ ]:
dataset = tf.keras.datasets.mnist.load_data(path="mnist.npz")
(train_x, train_y), (test_x, test_y) = dataset

In [ ]:
train_x_reordered = train_x.reshape((60000,-1))
print('TRAIN_X_REORDERED', train_x_reordered.shape)

test_x_reordered = test_x.reshape((10000,-1))
print('TEST_X_REORDERED', test_x_reordered.shape)


In [ ]:
mnist_dataset_train = ArrayMultiModalDataset(X=train_x_reordered, y=train_y, window_slices=[(0, 28*28)], 
                                             window_names=["px"])
mnist_dataset_test = ArrayMultiModalDataset(X=test_x_reordered, y=test_y, window_slices=[(0, 28*28)], 
                                             window_names=["px"])

In [ ]:
transform_tsne = TSNE()
transformer = TransformMultiModalDataset(transforms=[transform_tsne])
train_applied_tsne = transformer(mnist_dataset_train)
test_applied_tsne = transformer(mnist_dataset_test)

In [ ]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=60000)
metrics_train_applied_tsne = metrics_reporter.evaluate([mnist_dataset_train, train_applied_tsne])
print(metrics_train_applied_tsne)

In [ ]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=128)
metrics_train_applied_tsne = metrics_reporter.evaluate([mnist_dataset_train, train_applied_tsne])
print(metrics_train_applied_tsne)

In [ ]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=500)
metrics_train_applied_tsne = metrics_reporter.evaluate([mnist_dataset_train, train_applied_tsne])
print(metrics_train_applied_tsne)

In [ ]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=1000)
metrics_train_applied_tsne = metrics_reporter.evaluate([mnist_dataset_train, train_applied_tsne])
print(metrics_train_applied_tsne)

In [ ]:
transform_umap = UMAP()
transformer = TransformMultiModalDataset(transforms=[transform_umap])
train_applied_umap = transformer(mnist_dataset_train)
test_applied_umap = transformer(mnist_dataset_test)

In [ ]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=128)
metrics_train_applied_umap = metrics_reporter.evaluate([mnist_dataset_train, train_applied_umap])
print(metrics_train_applied_umap)

In [ ]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=500)
metrics_train_applied_umap = metrics_reporter.evaluate([mnist_dataset_train, train_applied_umap])
print(metrics_train_applied_umap)

In [ ]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=1000)
metrics_train_applied_umap = metrics_reporter.evaluate([mnist_dataset_train, train_applied_umap])
print(metrics_train_applied_umap)

In [ ]:
from librep.transforms.topo_ae import TopologicalDimensionalityReduction
from librep.estimators.ae.torch.models.topological_ae.topological_ae import TopologicallyRegularizedAutoencoder

In [ ]:
topo_transformer = TopologicallyRegularizedAutoencoder()
optimizer = torch.optim.Adam(topo_transformer.parameters(), lr=1e-3, weight_decay=1e-5)
# topo_transformer.fit(train_x)
# autoencoder_model='DeepAE'
data_loader = torch.utils.data.DataLoader(dataset=train_x, batch_size=128, shuffle=True)
for img in data_loader:
    orig = img
    reshaped = 
    print('IMG', img.shape)
    assert 1==0

In [ ]:
import torch
from torchvision import transforms as torch_transforms
import numpy as np
torch_transformer = torch_transforms.ToTensor()
X = train_x
patience = 10
max_loss = 1000
num_epochs = 1000
batch = 128
for epoch in range(num_epochs):
    topo_transformer.train()
    for img in data_loader:
        # print(img)
        # assert 1==0
        reshaped = np.reshape(img, (-1,1,28,28))
        reshaped = torch.Tensor(reshaped).float()
        loss, _ = topo_transformer(reshaped)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')
    if max_loss < loss.item():
        if patience == 0:
            break
        patience -= 1
    else:
        max_loss = loss.item()

In [ ]:
import torch
from torchvision import transforms as torch_transforms
import numpy as np
torch_transformer = torch_transforms.ToTensor()
X = train_x
patience = 10
max_loss = 1000
num_epochs = 1000
batch = 128
for epoch in range(num_epochs):
    topo_transformer.train()
    for i in range(0,len(X), batch):
        # row = np.array([X[i]])
        # print('row',row.shape)
        orig_value = np.array(X[i:i+batch])
        # print('ORIG_VAL', orig_value.shape)
        reshaped = np.reshape(orig_value, (-1,1, 28, 28))
        # print('RESHAPED', reshaped.shape)
        # print(np.array(X[20*i:20*i + 200]).shape)
        sample = torch.Tensor(reshaped)
        # sample = torch_transformer(reshaped)
        # torch.squeeze(sample)
        loss, _ = topo_transformer(sample)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')
    if max_loss < loss.item():
        if patience == 0:
            break
        patience -= 1
    else:
        max_loss = loss.item()

In [ ]:
# ae_model='DeepAE'
transform_topoae = TopologicalDimensionalityReduction()
transform_topoae.fit(train_x)

In [ ]:
transform_topoae.fit(mnist_dataset_train.X)

In [ ]:
transform_topoae.transform(mnist_dataset_test.X)

In [ ]:
transformer = TransformMultiModalDataset(transforms=[transform_topoae])
train_applied_topoae = transformer(mnist_dataset_train)
test_applied_topoae = transformer(mnist_dataset_test)

In [ ]:
trains_by_y = [[], [], [], [], [], [], [], [], [], []]

In [ ]:
# ADD TUTORIAL
import numpy as np
import pandas as pd

from librep.base.data import SimpleDataset
from librep.transforms import UMAP
from librep.datasets.multimodal import TransformMultiModalDataset, ArrayMultiModalDataset

In [ ]:
mnist_dataset_train = ArrayMultiModalDataset(X=train_x_reordered, y=train_y, window_slices=[(0, 28*28)], 
                                             window_names=["px"])
mnist_dataset_test = SimpleDataset(X=test_x_reordered, y=test_y)

In [ ]:
transform_umap = UMAP()
transformer = TransformMultiModalDataset(transforms=[transform_umap])
transformed_dataset = transformer(mnist_dataset_train)

In [ ]:
print(transformed_dataset.X.shape)

In [ ]:
transformer = TransformMultiModalDataset(transforms=[transform_umap])
transformed_dataset = transformer(mnist_dataset_train.X)


In [ ]:
mnist_dataset_train.X.shape

In [ ]:
transformer.transforms

In [ ]:
transform_umap = UMAP()
transformed_dataset = transform_umap.fit_transform(train_x_reordered)
# transformer = TransformMultiModalDataset(transforms=[transform_umap])
# transformed_dataset = transformer(mnist_dataset_train)
print(transformed_dataset)

In [ ]:
class MNISTDataset(Dataset):
    def __init__(self, X: list, y: list):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return (self.X[index], self.y[index])

    # (optional) Simple interface to describe the object as string
    def __str__(self) -> str:
        return f"MNISTDataset: {len(self)} samples; {len(set(self.y))} classes"

    def __repr__(self) -> str:
        return str(self)

In [ ]:
samples = [[1, 2, 3], [3, 2, 1], [0.5, 0.5, 0.5], [0, 0, 0]]
labels = [0, 0, 1, 1]

dataset = MNISTDataset(X=samples, y=labels)
dataset

In [ ]:
dataset[0]

In [ ]:
(train_x, train_y), (test_x, test_y) = dataset
trains_by_y = [[], [], [], [], [], [], [], [], [], []]

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
tsne_reducer = TSNE()
np_train_x = np.array(train_x)
print(np_train_x.shape)
reordered = np_train_x.reshape((60000,-1))
print(reordered.shape)
train_x_2d = tsne_reducer.fit_transform(np.array(reordered))

In [ ]:
for index in range(len(train_y)):
    y_val = train_y[index]
    trains_by_y[y_val].append(train_x_2d[index])
# print(trains_by_y[0])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(figsize=(10, 10), dpi=80)
for i in range(len(trains_by_y)):
    data = trains_by_y[i]
    plt.plot(np.array(data)[:,0], np.array(data)[:,1], '.', markersize=10, label=str(i))
plt.legend()

In [ ]:
from pathlib import Path  # For defining dataset Paths
import sys                # For include librep package

# This must be done if librep is not installed via pip,
# as this directory (examples) is appart from librep package root
sys.path.append("..")

# Third party imports
import pandas as pd
import numpy as np

# Librep imports
from librep.utils.dataset import PandasDatasetsIO          # For quick load train, test and validation CSVs
from librep.datasets.multimodal import PandasMultiModalDataset # Wrap CSVs to librep's `Dataset` interface

In [ ]:
# Path for KuHar balanced view with the same activities (and labels numbers) as MotionSense
# It is assumed that the directory will contain (train.csv, test.csv and validation.csv)
kuhar_dataset_path = Path("../data/views/KuHar/balanced_motionsense_equivalent_view")

# Path for Motionsese balanced view
motionsense_dataset_path = Path("../data/views/MotionSense/balanced_view")

In [ ]:
import numpy as np
data = np.random.random((100, 3))
y = np.concatenate([[1]*75, [2]*25])
print(data.shape, y.shape)
from sklearn.model_selection import train_test_split
train_X, val_X, train_Y, val_Y = train_test_split(data, y, random_state=42, train_size = 0.5, stratify=y)
print(np.count_nonzero(val_Y == 1), np.count_nonzero(val_Y == 2))